In [ ]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 62.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import pandas as pd
from z3 import *


from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor


In [ ]:
BOARD_SIZE = 900
IMG_SIZE = 28
SCALE_FACTOR = 2

##Loading in character recognition model

In [ ]:
class Grid_CNN(nn.Module):
    def __init__(self, output_dim):
        super(Grid_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(262144, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
transform=transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
])

In [ ]:
class CNN_v2(nn.Module):
    def __init__(self, output_dim):
        super(CNN_v2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(3136, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
character_model = CNN_v2(output_dim=14)

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/Summer2025Research/KenKenSolver/model/character_recognition_v2_model_weights.pth')

In [ ]:
character_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
character_model.eval()

CNN_v2(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=14, bias=True)
)

In [ ]:
grid_detection = Grid_CNN(output_dim=5)

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/Summer2025Research/KenKenSolver/model/grid_detection_model_weights.pth')

In [ ]:
grid_detection.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
grid_detection.eval()

Grid_CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=262144, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)

##Get size with Grid CNN

In [ ]:
def get_size(filename):
  im = Image.open(filename).convert("RGBA")
  im = transform(im).unsqueeze(0)
  output = grid_detection(im)
  prediction = torch.argmax(output, dim=1).item()
  return prediction + 3

##OpenCV grid + border detection

In [ ]:
def find_h_borders(h_lines, size, epsilon, delta):
  cell_size = ((BOARD_SIZE*SCALE_FACTOR)//size)
  vertical_window = (cell_size-delta, cell_size+delta)
  h_borders = np.zeros((size-1, size))

  horizontal_window = (epsilon, cell_size-epsilon)

  for i in range(size-1):
    window = h_lines[(h_lines['y1'] >= vertical_window[0]) & (h_lines['y1'] <= vertical_window[1])]
    max_val = (window[['y1', 'y2']].min(axis=1)).max()
    min_val = (window[['y1', 'y2']].max(axis=1)).min()

    if max_val - min_val > int(11 * SCALE_FACTOR):

      for j in range(size):

        y_vals = window[(np.maximum(window['x1'], window['x2'])>=horizontal_window[0]) & (np.minimum(window['x1'], window['x2'])<=horizontal_window[1])]['y1'].values

        if max_val in y_vals or min_val in y_vals:
          h_borders[i][j] = 1
        horizontal_window = (horizontal_window[0]+cell_size, horizontal_window[1]+cell_size)
      horizontal_window = (epsilon, cell_size-epsilon)

    vertical_window = (vertical_window[0]+cell_size, vertical_window[1]+cell_size)

  return h_borders


In [ ]:
def find_v_borders(v_lines, size, epsilon, delta):
  cell_size = ((BOARD_SIZE*SCALE_FACTOR)//size)
  horizontal_window = (cell_size-delta, cell_size+delta)
  v_borders = np.zeros((size, size-1))

  vertical_window = (epsilon, cell_size-epsilon)

  for i in range(size-1):
    window = v_lines[(v_lines['x1'] >= horizontal_window[0]) & (v_lines['x1'] <= horizontal_window[1])]
    max_val = (window[['x1', 'x2']].min(axis=1)).max()
    min_val = (window[['x1', 'x2']].max(axis=1)).min()

    if max_val - min_val > 11:
      for j in range(size):

        x_vals = window[(np.maximum(window['y1'], window['y2'])>=vertical_window[0]) & (np.minimum(window['y1'], window['y2'])<=vertical_window[1])]['x1'].values
        if max_val in x_vals or min_val in x_vals:
          v_borders[j][i] = 1
        vertical_window = (vertical_window[0]+cell_size, vertical_window[1]+cell_size)
      vertical_window = (epsilon, cell_size-epsilon)
    horizontal_window = (horizontal_window[0]+cell_size, horizontal_window[1]+cell_size)

  return v_borders

In [ ]:
def make_cage(start_cell, visited, h_borders, v_borders):
  cage = [start_cell]
  neighbors = [start_cell]
  visited[start_cell[0]][start_cell[1]] = 1

  while neighbors:
    for neighbor in neighbors:
      start_cell = neighbor
      neighbors = []
      if start_cell[1] > 0 and visited[start_cell[0]][start_cell[1]-1] == 0 and v_borders[start_cell[0]][start_cell[1]-1] == 0:
        cage.append([start_cell[0], start_cell[1]-1])
        visited[start_cell[0]][start_cell[1]-1] = 1
        neighbors.append([start_cell[0], start_cell[1]-1])

      if start_cell[0] > 0 and visited[start_cell[0]-1][start_cell[1]] == 0 and h_borders[start_cell[0]-1][start_cell[1]] == 0:
        cage.append([start_cell[0]-1, start_cell[1]])
        visited[start_cell[0]-1][start_cell[1]] = 1
        neighbors.append([start_cell[0]-1, start_cell[1]])

      if start_cell[1] < len(v_borders)-1 and visited[start_cell[0]][start_cell[1]+1] == 0 and v_borders[start_cell[0]][start_cell[1]] == 0:
        cage.append([start_cell[0], start_cell[1]+1])
        visited[start_cell[0]][start_cell[1]+1] = 1
        neighbors.append([start_cell[0], start_cell[1]+1])

      if start_cell[0] < len(v_borders)-1 and visited[start_cell[0]+1][start_cell[1]] == 0 and h_borders[start_cell[0]][start_cell[1]] == 0:
        cage.append([start_cell[0]+1, start_cell[1]])
        visited[start_cell[0]+1][start_cell[1]] = 1
        neighbors.append([start_cell[0]+1, start_cell[1]])
  return cage

In [ ]:
def construct_cages(h_borders, v_borders):
  size = len(v_borders)
  cages = []
  num_cages = 0
  visited = np.zeros((size, size))
  for row in range(size):
    for col in range(size):
      start_cell = [row, col]
      if visited[row][col] == 0:
        cages.append(make_cage(start_cell, visited, h_borders, v_borders))
  return cages

In [ ]:
def get_border_thickness(lines):
  v_lines = lines[lines['x1'] == lines['x2']]
  return min(v_lines['x1'])

In [ ]:
def find_size_and_borders(filename):

  #src = cv.imread(cv.samples.findFile(filename), cv.IMREAD_GRAYSCALE)
  src = cv.imread(filename)
  resized = cv.resize(src, (BOARD_SIZE * SCALE_FACTOR, BOARD_SIZE * SCALE_FACTOR))

  filtered = cv.pyrMeanShiftFiltering(resized, sp = 5, sr = 40)
  dst = cv.Canny(filtered, 50, 200, None, 3)
  cdstP = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
  linesP = cv.HoughLinesP(dst, 1, np.pi / 360, 75, None, 50, 15)
  linesP = np.squeeze(linesP, axis=1)
  lines_df = pd.DataFrame(linesP, columns=['x1', 'y1', 'x2', 'y2'])
  h_lines = lines_df[abs(lines_df['y1'] - lines_df['y2']) < 2]
  v_lines = lines_df[abs(lines_df['x1'] - lines_df['x2']) < 2]
  border_thickness = get_border_thickness(lines_df)

  # tmp = (BOARD_SIZE * SCALE_FACTOR) - max(v_lines[(v_lines['x1']<(BOARD_SIZE * SCALE_FACTOR)-(border_thickness)) & (v_lines[['y1', 'y2']].max(axis=1) > (BOARD_SIZE * SCALE_FACTOR)-(border_thickness*2))]['x1'].values)
  # size = round((BOARD_SIZE * SCALE_FACTOR) / tmp)

  size = get_size(filename)
  cages = construct_cages(find_h_borders(h_lines, size, border_thickness, border_thickness), find_v_borders(v_lines, size, border_thickness, border_thickness))
  return size, cages, border_thickness // SCALE_FACTOR

##Image Segmentation

In [ ]:
def get_contours(img):
  img = (img * 255).astype(np.uint8)
  _, inp = cv.threshold(img,127,255,cv.THRESH_BINARY_INV)

  e_kernel = np.ones((1, 1), np.uint8)
  inp = cv.erode(inp, e_kernel, iterations=1)
  d_kernel = np.ones((3, 3), np.uint8)
  inp = cv.dilate(inp, d_kernel, iterations=1)

  ctrs, hierarchy = cv.findContours(inp.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  ctrs = sorted(ctrs, key=lambda cnt: cv.boundingRect(cnt)[0])

  #merge overlapping boxes
  boxes = [cv.boundingRect(ctrs[0])]
  count =1
  while count < len(ctrs):
      x, y, w, h = boxes[-1]

      x2, y2, w2, h2 = cv.boundingRect(ctrs[count])
      if x2 < (x+w):
        h = abs(y - y2) + h2 if y < y2 else abs(y - y2) + h
        w = max(w, w2)
        y = min(y, y2)
        boxes[-1] = (x, y, w, h)

      else:
        boxes.append((x2, y2, w2, h2))
      count +=1

  return boxes

In [ ]:
def get_character(img, box):
    char = np.ones((IMG_SIZE, IMG_SIZE), dtype=np.float32)
    x, y, w, h = box
    cropped = img[y:y+h, x:x+w]

    cropped_img = Image.fromarray((cropped * 255).astype(np.uint8)).convert('L')

    aspect = w / h
    if aspect > 1:
        new_w = IMG_SIZE
        new_h = int(IMG_SIZE / aspect)
    else:
        new_h = IMG_SIZE
        new_w = int(IMG_SIZE * aspect)

    #print(new_w, new_h)

    resized_img = cropped_img.resize((new_w, new_h), Image.Resampling.LANCZOS)

    canvas = Image.new('L', (IMG_SIZE, IMG_SIZE), color=255)
    paste_x = (IMG_SIZE - new_w) // 2
    paste_y = (IMG_SIZE - new_h) // 2
    canvas.paste(resized_img, (paste_x, paste_y))

    result = np.array(canvas).astype(np.float32) / 255.0
    return result

In [ ]:
def segment_cell(grid, size, border_thickness, row, col):
  cell_size = len(grid) // size

  cell = grid[row*cell_size + border_thickness: row*cell_size + cell_size//2,
              col*cell_size + border_thickness: (col+1)*cell_size - border_thickness]

  cell = (cell / 255.0).astype('float64')
  contours = get_contours(cell)
  #print(contours)
  characters = []
  for box in contours:
    characters.append(get_character(cell, box))

  return characters

##Classifying symbols

In [ ]:
def get_predictions(characters):
  predictions = []
  #model.eval()
  with torch.no_grad():
    for c in characters:
      im = torch.tensor(c, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
      output = character_model(im)
      predictions.append(torch.argmax(output, dim=1).item())
  return predictions

##Building the symbolic representation

In [ ]:
def make_puzzle(size, border_thickness, cages, filename):
  img = Image.open(filename).convert('L')
  grid = np.array(img)
  puzzle = []
  for cage in cages:
    puzzle.append({"cells":cage, "op":"", "target": 0})
    characters = segment_cell(grid, size, border_thickness+5, cage[0][0], cage[0][1])
    predictions = get_predictions(characters)
    puzzle[-1] = update_puzzle(puzzle[-1], predictions)
  return puzzle



In [ ]:
def update_puzzle(puzzle, predictions):
  if len(predictions) == 1:
    puzzle["target"] = predictions[0]
  else:
    target = 0
    for i in range(len(predictions)-1):
      power = len(predictions)-2-i
      target += predictions[i] * (10 ** power)
    if predictions[-1] == 10:
      op = "add"
    elif predictions[-1] == 11:
      op = "div"
    elif predictions[-1] == 12:
      op = "mul"
    elif predictions[-1] == 13:
      op = "sub"
    puzzle["target"] = target
    puzzle["op"] = op
  return puzzle


##Add Z3 Solver

In [ ]:
def parse_block_constraints(puzzle, cells):
    constraints = []
    for block in puzzle:
        op = block["op"]
        target = block["target"]
        vars_in_block = [cells[i][j] for i, j in block["cells"]]
        if op == "":
            constraints.append(vars_in_block[0] == target)
        elif op == "add":
            constraints.append(Sum(vars_in_block) == target)
        elif op == "mul":
            product = vars_in_block[0]
            for v in vars_in_block[1:]:
                product *= v
            constraints.append(product == target)
        elif op == "sub" and len(vars_in_block) == 2:
            a, b = vars_in_block
            constraints.append(Or(a - b == target, b - a == target))
        elif op == "div" and len(vars_in_block) == 2:
            a, b = vars_in_block
            constraints.append(Or(a / b == target, b / a == target))
        else:
            raise ValueError(f"Unsupported operation or malformed block: {block}")
    return constraints



In [ ]:
def evaluate_puzzle(puzzle, size):
  X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(size) ]
      for i in range(size) ]
  cells_c  = [ And(1 <= X[i][j], X[i][j] <= size)
              for i in range(size) for j in range(size) ]
  rows_c   = [ Distinct(X[i]) for i in range(size) ]
  cols_c   = [ Distinct([ X[i][j] for i in range(size) ])
              for j in range(size) ]
  constraints = cells_c + rows_c + cols_c + parse_block_constraints(puzzle, X)
  instance = [[0] * size] * size
  instance = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(size) for j in range(size) ]
  s = Solver()

  #s.set("timeout", 70000)
  problem = constraints + instance
  s.add(problem)
  if s.check() == sat:
    m = s.model()
    solution = [ [ m.evaluate(X[i][j]) for j in range(size) ]
      for i in range(size) ]
    return solution
  else:
    print("failed to solve: constraints unsatisfiable")
    return None

##Full pipeline

In [ ]:
filename = "/content/drive/MyDrive/Summer2025Research/KenKenSolver/images/boards_noto_sans/board3_0.png"

In [ ]:
size, cages, border_thickness = find_size_and_borders(filename)
puzzle = make_puzzle(size, border_thickness, cages, filename)
solution = evaluate_puzzle(puzzle, size)

In [ ]:
def solve(filename):
  size, cages, border_thickness = find_size_and_borders(filename)
  puzzle = make_puzzle(size, border_thickness, cages, filename)
  solution = evaluate_puzzle(puzzle, size)
  return solution

In [ ]:
solution

[[3, 2, 1], [1, 3, 2], [2, 1, 3]]

In [ ]:
puzzle

[{'cells': [[0, 0], [0, 1], [1, 0]], 'op': 'mul', 'target': 6},
 {'cells': [[0, 2]], 'op': '', 'target': 1},
 {'cells': [[1, 1], [2, 1]], 'op': 'div', 'target': 3},
 {'cells': [[1, 2], [2, 2]], 'op': 'sub', 'target': 1},
 {'cells': [[2, 0]], 'op': '', 'target': 2}]

###Avg Time


*   3x3: 3s
*   4x4: 2s
*   5x5: 1s
*   6x6: 8s
*   7x7: 17s



##Testing on full Noto Sans boards dataset

Correct performance: solution found

In [ ]:
import time

In [ ]:
import json

with open("/puzzles/puzzles_dict.json", "r") as f:
    puzzles_ds = json.load(f)


In [ ]:
accuracy = {3:0, 4:0, 5:0, 6:0, 7:0}
avg_time = {3:0, 4:0, 5:0, 6:0, 7:0}
total = 0

In [ ]:
for size in range(3, 8):
  for i in range(0, len(puzzles_ds[str(size)])):
    filepath= "/board_images/board"+str(size)+"_"+str(i)+".png"
    start = time.time()
    s, cages, b_t = find_size_and_borders(filepath)
    try:
      puzzle = make_puzzle(s, b_t, cages, filepath)
      solution = evaluate_puzzle(puzzle, s)
      if solution:
        accuracy[size]+=1
    except:
      pass
    end = time.time()
    avg_time[size] += (end - start)
    total+=1
    print(str(accuracy[size])+"/"+str(total))
  avg_time[size] = avg_time[size] / total
  accuracy[size] = accuracy[size] / total
  total = 0

1/1
failed to solve: constraints unsatisfiable
1/2
failed to solve: constraints unsatisfiable
1/3


KeyboardInterrupt: 



*   100% 3x3 accuracy
*   100% 4x4 accuracy
*   100% 5x5 accuracy
*   100% 6x6 accuracy
*   100% 7x7 accuracy (20 mins for 30 puzzles)


##Saving Results

In [ ]:
results = pd.DataFrame({
    'accuracy (%)': accuracy,
    'avg_time (s)': avg_time
})

In [ ]:
results.to_csv('/results/neurosymbolic_solver.csv', index=False)

##Uploading to GitHub Repo

In [1]:
%cd /content/drive/MyDrive/Summer2025Research/SolverRepo

/content/drive/MyDrive/Summer2025Research/SolverRepo


In [2]:
!git config --global user.email "kierstenb39@gmail.com"
!git config --global user.name "Kiersten Brennan"

In [3]:
!git init
!git remote add origin https://github.com/KBrennan39/KenKenSolver.git

Reinitialized existing Git repository in /content/drive/MyDrive/Summer2025Research/SolverRepo/.git/
error: remote origin already exists.


In [ ]:
!git add NeuroSymbolicSolver.ipynb AnalyzingResults.ipynb

In [4]:
!git status

Refresh index: 100% (11/11), done.
On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   NeuroSymbolicSolver.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.env
	AnalyzingResults.ipynb
	ClaudeEvaluation.ipynb
	GPTEvaluation.ipynb
	GeminiEvaluation.ipynb
	results/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git push origin main

Uploading LFS objects: 100% (2/2), 136 MB | 13 MB/s, done.
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (7/7), 2.06 KiB | 95.00 KiB/s, done.
Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/KBrennan39/KenKenSolver.git
   518f99b..bbbe70b  main -> main
